# Real-Time Analytics com Apache Spark Streaming

<br>

### Objetivo
O objetivo deste Lab é trazer um exemplo de análise de dados em tempo real com o Spark Structured Streaming, demonstrando a arquitetura de funcionamento desse tipo de análise. O Lab também é uma introdução para os Mini-Projetos 6 e 7 do curso, quando, além de usar o Spark Streaming, trabalharemos com o Apache Kafka.

### Instruções

- Abra o Terminal e digite o seguinte comando: `nc -lk 9999`

Desta forma, você poderá digitar no terminal os comandos posteriores para simular a entrada de dados em tempo real.

- Agora, deixe essa janela do terminal aberta e abra outra janela do terminal.
- Nesta nova janela, conecte-se ao pyspark para buscar esses dados em tempo real.
> À medida que você digitar palavras no terminal que simula entradas de dados em tempo real, o processo de análise irá capturar e processar os dados, entregando o resultado em tempo real.

<br>

-  Neste exemplo vamos **analisar o número de ocorrências de palavras**. Queremos saber quantas vezes cada palavra aparece nesse streaming. Essa foi a lógica escolhida para este processo de análise.

> Este trecho de código foi extraído da própria documentação da biblioteca.

### Como Usar

- Rodar o script abaixo e digitar palavras ou frases no terminal e aguardar pela analise abaixo.

In [2]:
# Lab 6 - Análise de Dados em Tempo Real com Apache Spark Structured Streaming
# https://spark.apache.org/docs/latest/structured-streaming-programming-guide.html

# Funções
from pyspark.sql import SparkSession
from pyspark.sql.functions import explode, split, regexp_replace

# Sessão
spark = SparkSession.builder.appName("StructuredNetworkWordCount").getOrCreate()

# Criamos um DataFrame para o Stream que vai extrair os dados em tempo real do gerador de dados em localhost:9999
lines = spark.readStream.format("socket").option("host", "localhost").option("port", 9999).load()

# Limpa pontuações e divide cada linha de dados em palavras
cleaned_lines = lines.select(
    regexp_replace(lines.value, "[^\w\s]", "").alias("value")  # Remove pontuações
)

words = cleaned_lines.select(
    explode(split(cleaned_lines.value, " ")).alias("word")
)

# Contador de palavras
wordCounts = words.groupBy("word").count()

# Executa a query em tempo real e imprime a contagem de palavras em tempo real
query = wordCounts.writeStream.outputMode("complete").format("console").start()

# Aguarda término da conexão
query.awaitTermination()

24/08/19 18:38:37 WARN TextSocketSourceProvider: The socket source should not be used for production applications! It does not support recovery.
24/08/19 18:38:37 WARN ResolveWriteToStream: Temporary checkpoint location created which is deleted normally when the query didn't fail: /tmp/temporary-4463732e-2dcb-4d6c-a81c-749b33998ada. If it's required to delete it under any circumstances, please set spark.sql.streaming.forceDeleteTempCheckpointLocation to true. Important to know deleting temp checkpoint folder is best effort.
24/08/19 18:38:37 WARN ResolveWriteToStream: spark.sql.adaptive.enabled is not supported in streaming DataFrames/Datasets and will be disabled.


-------------------------------------------
Batch: 0
-------------------------------------------
+----+-----+
|word|count|
+----+-----+
+----+-----+



-------------------------------------------
Batch: 1
-------------------------------------------
+--------+-----+
|    word|count|
+--------+-----+
|palavras|    3|
|     sao|    1|
|   agora|    1|
|testando|    1|
|      se|    1|
|      ou|    1|
+--------+-----+



-------------------------------------------
Batch: 2
-------------------------------------------
+-----------+-----+
|       word|count|
+-----------+-----+
|   palavras|    3|
|        sao|    1|
|sensasional|    1|
|      agora|    1|
|   testando|    1|
|         se|    1|
|         ou|    1|
+-----------+-----+



24/08/19 18:39:18 WARN TextSocketMicroBatchStream: Stream closed by localhost:9999
ERROR:root:KeyboardInterrupt while sending command.
Traceback (most recent call last):
  File "/home/eduardo/anaconda3/lib/python3.9/site-packages/py4j/java_gateway.py", line 1038, in send_command
    response = connection.send_command(command)
  File "/home/eduardo/anaconda3/lib/python3.9/site-packages/py4j/clientserver.py", line 511, in send_command
    answer = smart_decode(self.stream.readline()[:-1])
  File "/home/eduardo/anaconda3/lib/python3.9/socket.py", line 704, in readinto
    return self._sock.recv_into(b)
KeyboardInterrupt


KeyboardInterrupt: 

### Executando o Código no Jupyter Notebook

- Abra o Jupyter Notebook: Inicie o Jupyter Notebook e abra um novo notebook.

- Cole o código acima: Em uma célula de código do notebook, cole o código fornecido.

- Execute a célula: Clique em "Run" (ou pressione Shift + Enter) para executar a célula.

- Simular dados de streaming: Em um terminal separado, execute o comando nc -lk 9999 para iniciar o gerador de dados. Você pode então começar a digitar palavras para ver a contagem de palavras sendo atualizada em tempo real no notebook.